<a href="https://colab.research.google.com/github/LastBitcoiner/OpenFaceAI/blob/main/Face_age_detection_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# start time 4:58:00 Tehran Time 08/08/2021
# we need to classification age and gender by photos

In [3]:
# connect to github

!ssh-keygen -t ed25519 -C "code.omidsha@gmail.com"

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): 
Created directory '/root/.ssh'.
Enter passphrase (empty for no passphrase): 1212
Enter same passphrase again: 1212
Your identification has been saved in /root/.ssh/id_ed25519.
Your public key has been saved in /root/.ssh/id_ed25519.pub.
The key fingerprint is:
SHA256:g82IpWESaiouiARm0toWsv6ANknKcihNaGdf2fO8Vkc code.omidsha@gmail.com
The key's randomart image is:
+--[ED25519 256]--+
|  .              |
| o .             |
|=++ o .          |
|B* + = =o      E |
|*++oo ooSo    .  |
|&=+ . .  .+  . . |
|XOo  .     o. .  |
|=.+        ..    |
|   .      ..     |
+----[SHA256]-----+


## download dataset from kaggle
## init kaggle configuration from api file


In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"omidshahsavarzadeh","key":"01d7f900b98709315926594706249938"}'}

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nipunarora8/age-gender-and-ethnicity-face-data-csv

 84% 53.0M/63.2M [00:03<00:01, 7.67MB/s]
100% 63.2M/63.2M [00:03<00:00, 20.6MB/s]


## Prepare dataset for train

In [ ]:
!unzip /content/drive/MyDrive/Face_detect/age-gender-and-ethnicity-face-data-csv.zip

Archive:  /content/drive/MyDrive/Face_detect/age-gender-and-ethnicity-face-data-csv.zip
  inflating: age_gender.csv          


In [ ]:
# imports for data works
import pandas as pd
import numpy as np

# imports for plotting
import matplotlib.pyplot as plt
import plotly.express as px

# for create model
from tensorflow.keras import layers, models
import tensorflow as tf

In [ ]:
# load dataset
csv_data = '/content/drive/MyDrive/Face_detect/age_gender.csv'
df = pd.read_csv(csv_data)


# drop useless column
df.drop(columns={'img_name'},inplace=True)


# Categories
ethnicity_list = ["White", "Black", "Asian", "Indian", "Hispanic"]
gender_list = ['Male', 'Female']
age_list = ['Baby','Child','Adult','Elderly']

In [ ]:
"""
From the code above you can see that the bins are:

0 to 2 = ‘Toddler/Baby’
3 to 17 = ‘Child’
18 to 65 = ‘Adult’
above 65 =’Elderly’
"""

# append age range to the df
df['age_bins'] = pd.cut(df['age'],bins=[0,2,17,65,150],labels=range(4))
df.head()

In [ ]:
# convert pixels to 0-1 range
df['pixels']=df['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

df['pixels'] = df['pixels']/255

# define X and reshape it to picture size 
X = np.array(df['pixels'].tolist())
X = X.reshape(-1,48,48)

df['pixel_reshaped'] = X.tolist()
df.drop(columns={'pixels'},inplace=True)

In [ ]:
# calculating distributions
age_dist = df['age_bins'].value_counts()
age_dist.index = age_list
ethnicity_dist = df['ethnicity'].value_counts()
ethnicity_dist.index = ethnicity_list
gender_dist = df['gender'].value_counts()
gender_dist.index = gender_list

In [ ]:
# plot Age Distribution
px.bar(x=age_dist.index, y=age_dist.values, title='Age Distribution',
       labels={'x':'age','y':'count'}, width=800, height=400)

In [ ]:
# plot Ethnicity Distribution
px.bar(x=ethnicity_dist.index, y=ethnicity_dist.values, title='Ethnicity Distribution',
       labels={'x':'Ethnicity','y':'count'}, width=800, height=400)

In [ ]:
# Gender Distribution
px.bar(x=gender_dist.index, y=gender_dist.values, title='Gender Distribution',
       labels={'x':'Gender','y':'count'}, width=800, height=400)

In [ ]:
# plot sample images
plt.figure(figsize=(14,14))
for i in range(3720,3736):
  plt.subplot(4,4,(i%16)+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(X[i].squeeze(),cmap='gray')
  plt.title(ethnicity_list[df['ethnicity'].iloc[i]]
            + " " + gender_list[df['gender'].iloc[i]] + " Age:" + str(df['age'].iloc[i]))

In [ ]:
# train, validation and test dataset split function

def get_dataset_partitions_pd(df, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1
    
    # Only allows for equal validation and test splits
    assert val_split == test_split 

    # Specify seed to always have the same split distribution between runs
    df_sample = df.sample(frac=1, random_state=12)
    
    indices_or_sections = [int(train_split * len(df)), int((val_split + train_split) * len(df))]
    train_ds, val_ds, test_ds = np.split(df_sample, indices_or_sections)
    
    return train_ds, val_ds, test_ds

In [ ]:
# split train validate and test ds
train_ds, val_ds, test_ds = get_dataset_partitions_pd(df)

In [ ]:
df.head()

,age,ethnicity,gender,pixel_reshaped,age_bins
0,1,2,0,"[[0.5058823823928833, 0.501960813999176, 0.501...",0
1,1,2,0,"[[0.6431372761726379, 0.29019609093666077, 0.4...",0
2,1,2,0,"[[0.26274511218070984, 0.27450981736183167, 0....",0
3,1,2,0,"[[0.7568627595901489, 0.772549033164978, 0.776...",0
4,1,2,0,"[[0.7921568751335144, 0.8039215803146362, 0.81...",0


In [ ]:
# create diffrent labels
age_labels = np.array(df['age_bins'])
gender_labels = np.array(df['gender'])
ethnicity_labels = np.array(df[['ethnicity']])

In [ ]:
# create CNN model with Keras
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))



In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
Total params: 56,320
Trainable params: 56,320
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])